<a href="https://colab.research.google.com/github/ethanrom/LSTM-Sinhala-Text-Generation/blob/main/sinhala_news_titles_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import numpy as np 

In [ ]:
!gdown 1R8ZRTPzlqjxfrWdqA6Me3gsJwf2AjHnp

Downloading...
From: https://drive.google.com/uc?id=1R8ZRTPzlqjxfrWdqA6Me3gsJwf2AjHnp
To: /content/input.txt
100% 3.07M/3.07M [00:00<00:00, 152MB/s]


In [ ]:
# read the input file and store the poems as a list of strings
with open("input.txt", "r") as file:
    text = file.read().split("\n")

In [ ]:
# split the poems into 4 lines each and store in a new list
lines = []
for poem in text:
    poem = poem.split("\n")
    for line in poem:
        lines.append(line)

In [ ]:
# build the tokenizer on the lines
tokenizer = Tokenizer()

def tokenize(lines):
    # fit the tokenizer on the lines
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
# convert the lines to sequences
def get_sequence(tokenizer, max_length, lines):
    sequences = tokenizer.texts_to_sequences(lines)
    sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
    return sequences

In [ ]:
# get the vocabulary size
def get_vocabulary_size(tokenizer):
    return len(tokenizer.word_index) + 1

In [ ]:
#generate lines
def generate_text(model, tokenizer, max_length, seed_text, num_gen_words, temperature=1.5):
    # Initialize the generated text
    generated_text = seed_text
    # Convert the seed text to numerical representation
    input_sequence = [tokenizer.word_index[word] for word in seed_text.split()]
    input_sequence = pad_sequences([input_sequence], maxlen=max_length, padding='pre')
    # Get the prediction for the seed text
    preds = model.predict(input_sequence, verbose=0)[0]
    # Set the temperature
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # Sample the next word using the temperature-modified predictions
    next_index = np.argmax(np.random.multinomial(1, preds, 1))
    next_word = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(next_index)]
    generated_text += " " + next_word
    # Generate the remaining words
    line_count = 1
    for i in range(num_gen_words-1):
        input_sequence = [tokenizer.word_index[word] for word in generated_text.split()[-max_length:]]
        input_sequence = pad_sequences([input_sequence], maxlen=max_length, padding='pre')
        preds = model.predict(input_sequence, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_index = np.argmax(np.random.multinomial(1, preds, 1))
        next_word = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(next_index)]
        generated_text += " " + next_word
        if next_word == ".":
            line_count += 2
        if line_count == 1:
            break
    return generated_text

In [ ]:
# Generate lines no line limit
def generate_text(model, tokenizer, max_length, seed_text, num_gen_words, temperature=1.5):
    # Initialize the generated text
    generated_text = seed_text
    # Convert the seed text to numerical representation
    input_sequence = [tokenizer.word_index[word] for word in seed_text.split()]
    input_sequence = pad_sequences([input_sequence], maxlen=max_length, padding='pre')
    # Get the prediction for the seed text
    preds = model.predict(input_sequence, verbose=0)[0]
    # Set the temperature
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # Sample the next word using the temperature-modified predictions
    next_index = np.argmax(np.random.multinomial(1, preds, 1))
    next_word = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(next_index)]
    generated_text += " " + next_word
    # Generate the remaining words
    for i in range(num_gen_words-1):
        input_sequence = [tokenizer.word_index[word] for word in generated_text.split()[-max_length:]]
        input_sequence = pad_sequences([input_sequence], maxlen=max_length, padding='pre')
        preds = model.predict(input_sequence, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_index = np.argmax(np.random.multinomial(1, preds, 1))
        next_word = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(next_index)]
        generated_text += " " + next_word
    return generated_text


In [ ]:
# set the hyperparameters
max_length = 4 
num_gen_words = 4 

In [ ]:
# get the tokenizer
tokenizer = tokenize(lines)

In [ ]:
# get the vocabulary size
vocabulary_size = get_vocabulary_size(tokenizer)

In [ ]:
# get the sequences
sequences = get_sequence(tokenizer, max_length, lines)

In [ ]:
# split the sequences into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

In [ ]:
# one hot encode the target
y = tf.keras.utils.to_categorical(y, num_classes=vocabulary_size)

In [ ]:
#split the data into train and test sets
total_data = X.shape[0]
split_point = int(total_data * 0.8)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

In [ ]:
#define the model option 1
model = Sequential()
model.add(Embedding(vocabulary_size, 10, input_length=max_length - 1))
model.add(LSTM(50))
model.add(Dropout(0.1))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

In [ ]:
#define the model option 2
model = Sequential()
model.add(Embedding(vocabulary_size, 200, input_length=max_length-1))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 4, 200)            4447600   
                                                                 
 bidirectional_4 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 100)               20100     
                                                                 
 dense_9 (Dense)             (None, 22238)             2246038   
                                                                 
Total params: 6,954,538
Trainable params: 6,954,538
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#define the model option 3
model = Sequential()
model.add(Embedding(vocabulary_size, 100, input_length=max_length-1))
model.add(LSTM(100))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 7, 100)            54900     
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 549)               55449     
                                                                 
Total params: 190,749
Trainable params: 190,749
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [ ]:
# Create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, 64, input_length=max_length))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 5, 64)             1423232   
                                                                 
 bidirectional_2 (Bidirectio  (None, 40)               13600     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 32)                1312      
                                                                 
 dense_5 (Dense)             (None, 22238)             733854    
                                                                 
Total params: 2,171,998
Trainable params: 2,171,998
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Save the trained model
model.save("model.h5")

In [ ]:
#set the early stopping criteria alt
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

In [ ]:
#fit the model alt
model.fit(X, y, batch_size=32, epochs=20, verbose=1, callbacks=[earlystop])

Epoch 1/20
823/826 [============================>.] - ETA: 0s - loss: 6.9158 - accuracy: 0.0370

826/826 [==============================] - 14s 13ms/step - loss: 6.9156 - accuracy: 0.0369
Epoch 2/20
823/826 [============================>.] - ETA: 0s - loss: 5.7124 - accuracy: 0.1201

826/826 [==============================] - 11s 13ms/step - loss: 5.7125 - accuracy: 0.1204
Epoch 3/20
823/826 [============================>.] - ETA: 0s - loss: 4.8265 - accuracy: 0.2223

826/826 [==============================] - 11s 13ms/step - loss: 4.8244 - accuracy: 0.2226
Epoch 4/20
823/826 [============================>.] - ETA: 0s - loss: 4.0718 - accuracy: 0.3092

826/826 [==============================] - 11s 13ms/step - loss: 4.0726 - accuracy: 0.3092
Epoch 5/20
823/826 [============================>.] - ETA: 0s - loss: 3.4320 - accuracy: 0.3855

...

826/826 [==============================] - 11s 13ms/step - loss: 0.3114 - accuracy: 0.9097
Epoch 17/20
823/826 [============================>.] - ETA: 0s - loss: 0.2667 - accuracy: 0.9243

826/826 [==============================] - 11s 13ms/step - loss: 0.2670 - accuracy: 0.9241
Epoch 18/20
825/826 [============================>.] - ETA: 0s - loss: 0.2318 - accuracy: 0.9321

826/826 [==============================] - 11s 13ms/step - loss: 0.2317 - accuracy: 0.9322
Epoch 19/20
823/826 [============================>.] - ETA: 0s - loss: 0.1947 - accuracy: 0.9408

826/826 [==============================] - 11s 13ms/step - loss: 0.1945 - accuracy: 0.9409
Epoch 20/20
822/826 [============================>.] - ETA: 0s - loss: 0.1766 - accuracy: 0.9459

826/826 [==============================] - 11s 13ms/step - loss: 0.1767 - accuracy: 0.9457


In [ ]:
#evaluate the model alt
_, accuracy = model.evaluate(X, y, verbose=0)
print("Accuracy: %.2f" % (accuracy*100))

Accuracy: 97.95


In [ ]:
#generate a title

seed_text = "ජනපතිට"
generated_title = generate_text(model, tokenizer, max_length-1, seed_text, num_gen_words)
print("Generated Title:")
print(generated_title)

Generated Title:
ජනපතිට වාරණයක් හමුවේ කණ්ඩායමක් ඉවතට


<ipython-input-95-88c1599a1727>:12: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature
<ipython-input-95-88c1599a1727>:25: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


In [ ]:
seed_text = "ට්‍රම්ප්"
generated_title = generate_text(model, tokenizer, max_length-1, seed_text, num_gen_words, temperature=0.5)
print("Generated Title:")
print(generated_title)

Generated Title:
ට්‍රම්ප් නවසීලන්තයට යයි ඡායාරූප සහිතයි


<ipython-input-95-88c1599a1727>:25: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil
colab_link = "/content/model.h5"
gdrive_link = "/content/gdrive/MyDrive/ffd"
shutil.copy(colab_link, gdrive_link)

'/content/gdrive/MyDrive/ffd/model.h5'